In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
from tensorflow import keras

max_features = 10000  # 상위 만개의 단어들만을 사용합니다.
maxlen = 500  # 각 문서의 길이를 500으로 맞춥니다.

In [3]:
(x_train, y_train), (x_test, y_test) = keras.datasets.imdb.load_data(num_words=max_features)
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

In [4]:
from tensorflow.keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [5]:
from tensorflow.keras import layers
from tensorflow.keras import models

In [6]:
model = models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=maxlen))
model.add(layers.Bidirectional(layers.LSTM(64), merge_mode='concat'))
model.add(layers.Dense(2, activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 128)          1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense (Dense)               (None, 2)                 258       
                                                                 
Total params: 1,379,074
Trainable params: 1,379,074
Non-trainable params: 0
_________________________________________________________________


In [6]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

In [7]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
checkpoint_filepath = './temp/checkpoint_bi_lstm_en'
mc = ModelCheckpoint(checkpoint_filepath, monitor='val_loss', mode='min', 
                     save_weights_only=True, save_best_only=True)

In [8]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(learning_rate=0.0005), loss='binary_crossentropy', metrics='accuracy')

In [9]:
history=model.fit(x_train, y_train_one_hot, epochs=20, batch_size=128, validation_split=0.1, callbacks=[es, mc])

Epoch 1/20
176/176 [==============================] - 469s 3s/step - loss: 0.5001 - accuracy: 0.7651 - val_loss: 0.3752 - val_accuracy: 0.8464
Epoch 2/20
176/176 [==============================] - 488s 3s/step - loss: 0.2862 - accuracy: 0.8912 - val_loss: 0.2756 - val_accuracy: 0.8900
Epoch 3/20
176/176 [==============================] - 496s 3s/step - loss: 0.2241 - accuracy: 0.9158 - val_loss: 0.2755 - val_accuracy: 0.8872
Epoch 4/20
176/176 [==============================] - 493s 3s/step - loss: 0.1953 - accuracy: 0.9299 - val_loss: 0.2923 - val_accuracy: 0.8828
Epoch 5/20
176/176 [==============================] - 492s 3s/step - loss: 0.1729 - accuracy: 0.9381 - val_loss: 0.2878 - val_accuracy: 0.8920
Epoch 6/20
176/176 [==============================] - 491s 3s/step - loss: 0.1580 - accuracy: 0.9435 - val_loss: 0.3256 - val_accuracy: 0.8500
Epoch 7/20
176/176 [==============================] - 486s 3s/step - loss: 0.1403 - accuracy: 0.9496 - val_loss: 0.4914 - val_accuracy: 0.8636

In [10]:
model.evaluate(x_test, y_test_one_hot)

782/782 [==============================] - 44s 56ms/step - loss: 0.3731 - accuracy: 0.8690


[0.37307679653167725, 0.8690400123596191]

## Stacked bidirectional LSTM

아래와 같이 stacked bidirectional LSTM을 구현할 수 있습니다. <br>
이때, return_sequences=True로 설정합니다. 

In [7]:
model1 = models.Sequential()
model1.add(layers.Embedding(max_features, 128, input_length=maxlen))
model1.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True)))
model1.add(layers.Bidirectional(layers.LSTM(64)))
model1.add(layers.Dense(2, activation = 'softmax'))
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 500, 128)         98816     
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
Total params: 1,477,890
Trainable params: 1,477,890
Non-trainable params: 0
_________________________________________________________________


In [10]:
model1.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model1.fit(x_train, y_train_one_hot, batch_size=32, epochs=2, validation_split=0.2)

Epoch 1/2
782/782 [==============================] - 47s 61ms/step - loss: 0.3833 - accuracy: 0.8268 - val_loss: 0.3378 - val_accuracy: 0.8618
Epoch 2/2
782/782 [==============================] - 46s 59ms/step - loss: 0.2087 - accuracy: 0.9218 - val_loss: 0.3562 - val_accuracy: 0.8607


In [11]:
model1.evaluate(x_test, y_test_one_hot)

782/782 [==============================] - 11s 14ms/step - loss: 0.3562 - accuracy: 0.8607


[0.3561634421348572, 0.8607199788093567]